In [38]:
import praw
import pandas as pd
import os
import datetime as dt
import time

In [2]:
reddit = praw.Reddit(client_id='460necGe04OkzTt0i9cWGg', client_secret='zuvpweGNzmGaCUjrVNDi2JyQ36LEPg', user_agent='emphathy_aita')


In [3]:
trauma = '../../../../local/reddit_demographic/trauma/'
comment_path = trauma + 'comments/'
os.chdir(trauma)
os.listdir()


['data_20000_domesticviolence.csv',
 'data_10000_survivorsofabuse.csv',
 'data_10000_traumatoolbox.csv',
 'data_10000_selfharm.csv',
 'data_20000_traumatoolbox.csv',
 'data_10000_Anger',
 'data_20000_rapecounceling.csv',
 'data_10000_domesticviolence.csv',
 'data_10000_ptsd.csv',
 'data_20000_selfharm.csv',
 'data_1000_Anger.csv',
 'data_10000_rape.csv',
 'data_10000_rapecounceling.csv',
 'data_20000_StopSelfHarm.csv',
 'data_100_Anger.csv',
 'all_trauma_data.csv',
 'comments',
 'data_20000_ptsd.csv',
 'data_10000_StopSelfHarm.csv',
 'data_20000_MenGetRapedToo.csv',
 'data_20000_rape.csv',
 'data_20000_survivorsofabuse.csv',
 'data_10000_MenGetRapedToo.csv',
 'data_10000_Anger.csv']

In [17]:
df = pd.read_csv('all_trauma_data.csv', index_col = None)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.head()

,id,user,gender,title,body,debug
0,txteko,NotyourangeLbabe,F,Am I delusional?,"I (26f) feel like every time I reach for help,...",{I (26f)} feel like every time I reach for hel...
1,txsbpz,Ok-Pop-1123,F,I finally left,"Hey everyone! I (28F) finally left my toxic, a...","Hey everyone! {I (28F)} finally left my toxic,..."
2,twu1on,girl-with-one-leg,F,I [24F] reported him [25M] to the police,I reported my ex-boyfriend to the police after...,{I [24F]} reported him [25M] to the police
3,twbepm,GradeOk8691,M,as i am a male victim of domrstic violence,[removed],as {i am a male} victim of domrstic violence
4,tud2u2,Ok_Requirement_9394,F,My partner (32M) has anger issues I (22F) can'...,We've been together for two years. I can't han...,My partner (32M) has anger issues {I (22F)} ca...


In [18]:
submissions_csv_path = 'trauma' + '-submissions.csv'

In [56]:
start_time = time.time()
submissions_data_dict = {}
for row in df[:5].itertuples():
    i = row[1]
    submissions_dict = {
        "id" : [],
        "score" : [],
        "num_comments": [],
        "created_utc" : [],
        'subreddit':[]
    }
    
    submission_praw = reddit.submission(id=i)

    submissions_dict["id"] = submission_praw.id
    submissions_dict["score"] = submission_praw.score
    submissions_dict["num_comments"] = submission_praw.num_comments
    submissions_dict["created_utc"] = submission_praw.created_utc
    submissions_dict["subreddit"] = str(submission_praw.subreddit)


    submission_comments_csv_path = 'trauma' + '-submission_' + i + '-comments.csv'
    submission_comments_dict = {
        "comment_id" : [],
        "comment_parent_id" : [],
        "comment_body" : [],
        "comment_link_id" : [],
    }

    submission_praw.comments.replace_more(limit=None)
    # for each comment in flattened comment tree
    for comment in submission_praw.comments.list():
        submission_comments_dict["comment_id"].append(comment.id)
        submission_comments_dict["comment_parent_id"].append(comment.parent_id)
        submission_comments_dict["comment_body"].append(comment.body)
        submission_comments_dict["comment_link_id"].append(comment.link_id)

    comment_df = pd.DataFrame(submission_comments_dict)
    comment_df.to_csv(comment_path + '/' + submission_comments_csv_path,
                                                          index=False)
    # pd.DataFrame(submissions_dict).to_csv(comment_path + '/' + submissions_csv_path,
    #                                             index=False)    

    submissions_data_dict[i] = submissions_dict

# action = f"\t\t[Info] Found submissions: {pd.DataFrame(submissions_dict).shape[0]}"
# print(action)

# action = f"\t\t[Info] Elapsed time: {time.time() - start_time: .2f}s"
# print(action)                                            

In [58]:
submissions_data_dict

{'txteko': {'id': 'txteko',
  'score': 1,
  'num_comments': 5,
  'created_utc': 1649271425.0,
  'subreddit': 'domesticviolence'},
 'txsbpz': {'id': 'txsbpz',
  'score': 25,
  'num_comments': 9,
  'created_utc': 1649268486.0,
  'subreddit': 'domesticviolence'},
 'twu1on': {'id': 'twu1on',
  'score': 4,
  'num_comments': 6,
  'created_utc': 1649162510.0,
  'subreddit': 'domesticviolence'},
 'twbepm': {'id': 'twbepm',
  'score': 1,
  'num_comments': 0,
  'created_utc': 1649102319.0,
  'subreddit': 'domesticviolence'},
 'tud2u2': {'id': 'tud2u2',
  'score': 6,
  'num_comments': 3,
  'created_utc': 1648883963.0,
  'subreddit': 'domesticviolence'}}

In [57]:
pd.DataFrame(submissions_data_dict).T

,id,score,num_comments,created_utc,subreddit
txteko,txteko,1,5,1649271425.0,domesticviolence
txsbpz,txsbpz,25,9,1649268486.0,domesticviolence
twu1on,twu1on,4,6,1649162510.0,domesticviolence
twbepm,twbepm,1,0,1649102319.0,domesticviolence
tud2u2,tud2u2,6,3,1648883963.0,domesticviolence


In [44]:
os.listdir('comments')

['trauma-submissions.csv',
 'trauma-submission_tsw0yu-comments.csv',
 'trauma-submission_twu1on-comments.csv',
 'trauma-submission_tud2u2-comments.csv',
 'trauma-submission_txteko-comments.csv',
 'trauma-submission_ttj9xr-comments.csv',
 'trauma-submission_twbepm-comments.csv',
 'trauma-submission_tr6amq-comments.csv',
 'trauma-submission_tsby6v-comments.csv',
 'trauma-submission_ttd02u-comments.csv',
 'trauma-submission_txsbpz-comments.csv']

In [59]:
for i in os.listdir('comments'):
    sample = pd.read_csv(comment_path + i)
    print(len(sample))

1
4
6
3
5
2
0
9
8


IsADirectoryError: [Errno 21] Is a directory: '../../../../local/reddit_demographic/trauma/comments/log'

In [19]:
import pprint

In [22]:
submission = reddit.submission(id=df['id'][0])

pprint.pprint(vars(submission))

str(submission.subreddit)

{'_comments_by_id': {},
 '_fetched': False,
 '_reddit': <praw.reddit.Reddit object at 0x7f2f0c27f610>,
 'comment_limit': 2048,
 'comment_sort': 'confidence',
 'id': 'txteko'}


'domesticviolence'

In [12]:
comment_df2 = pd.DataFrame(submissions_dict)
comment_df2

ValueError: All arrays must be of the same length

In [ ]:
submission_comments_dict = {
"comment_id" : [],
"comment_body" : [],
"comment_link_id" : [],
}

In [ ]:
submission_praw.comments.replace_more(limit=None)
# for each comment in flattened comment tree
for comment in submission_praw.comments.list():
    submission_comments_dict["comment_id"].append(comment.id)
    submission_comments_dict["comment_parent_id"].append(comment.parent_id)
    submission_comments_dict["comment_body"].append(comment.body)
    submission_comments_dict["comment_link_id"].append(comment.link_id)